<a href="https://colab.research.google.com/github/Gibsdevops/Machine-learning-crash-course-/blob/main/rice_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

In [4]:
rice = pd.read_csv('Rice_Cammeo_Osmancik.csv')


In [6]:
rice.head()

,Area,Perimeter,Major_Axis_Length,Minor_Axis_Length,Eccentricity,Convex_Area,Extent,Class
0,15231,525.578979,229.749878,85.093788,0.928882,15617,0.572896,Cammeo
1,14656,494.311005,206.020065,91.730972,0.895405,15072,0.615436,Cammeo
2,14634,501.122009,214.106781,87.768288,0.912118,14954,0.693259,Cammeo
3,13176,458.342987,193.337387,87.448395,0.891861,13368,0.640669,Cammeo
4,14688,507.166992,211.743378,89.312454,0.906691,15262,0.646024,Cammeo


In [7]:
rice.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3810 entries, 0 to 3809
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Area               3810 non-null   int64  
 1   Perimeter          3810 non-null   float64
 2   Major_Axis_Length  3810 non-null   float64
 3   Minor_Axis_Length  3810 non-null   float64
 4   Eccentricity       3810 non-null   float64
 5   Convex_Area        3810 non-null   int64  
 6   Extent             3810 non-null   float64
 7   Class              3810 non-null   object 
dtypes: float64(5), int64(2), object(1)
memory usage: 238.3+ KB


In [13]:
X = rice[['Area', 'Perimeter', 'Major_Axis_Length', 'Minor_Axis_Length', 'Eccentricity', 'Convex_Area', 'Extent']]
#X = rice.drop('class', axis=1) bothe methods are the same

y = rice['Class']

In [22]:
#let's first know how many classes we have
print("the number of classes are: ", y.nunique())
print("The classes are: ", y.unique())

the number of classes are:  2
The classes are:  ['Cammeo' 'Osmancik']


In [27]:
#True for all Cammeo, False for all other digits.
y_train_Cammeo = (y == 'Cammeo')

print(y_train_Cammeo)

0        True
1        True
2        True
3        True
4        True
        ...  
3805    False
3806    False
3807    False
3808    False
3809    False
Name: Class, Length: 3810, dtype: bool


In [28]:
#splitting the data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y_train_Cammeo, random_state=42, test_size=0.2)

In [33]:
#feature scaling
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)

In [40]:
#SELECTING THE RANDOM FOREST CLASSIFIER FOR CLASSIFICATION
from sklearn.ensemble import RandomForestClassifier

rnd_clf = RandomForestClassifier(random_state=42)


In [45]:
#evaluating the forest performance

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

skfolds = StratifiedKFold(n_splits=3)

scores = cross_val_score(rnd_clf, X_train_scaled, y_train, cv=skfolds, scoring="accuracy")
print(scores)
print(scores.mean())


[0.92716535 0.92814961 0.92322835]
0.9261811023622047


In [49]:
#fine tune my model

from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [10, 50, 100, 200],
    'max_depth': [5, 10, 15, 20],
    'bootstrap': [True, False]
}

grid_search = GridSearchCV(rnd_clf, param_grid, cv=skfolds, scoring='accuracy')
param = grid_search.fit(X_train_scaled, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=False),
             estimator=RandomForestClassifier(max_depth=5, random_state=42),
             param_grid={'bootstrap': [True, False],
                         'max_depth': [5, 10, 15, 20],
                         'n_estimators': [10, 50, 100, 200]},
             scoring='accuracy')

GridSearchCV(cv=StratifiedKFold(n_splits=3, random_state=None, shuffle=False),
             estimator=RandomForestClassifier(max_depth=5, random_state=42),
             param_grid={'bootstrap': [True, False],
                         'max_depth': [5, 10, 15, 20],
                         'n_estimators': [10, 50, 100, 200]},
             scoring='accuracy')

In [50]:
#display all the validation results
#the results of the gridsearch are stored in the cv_results_ variable

results = grid_search.cv_results_

for mean_score, params in zip(results['mean_test_score'], results['params']):
  print(f"Mean score: {mean_score}, parameters: {params}")

Mean score: 0.9225721784776902, parameters: {'bootstrap': True, 'max_depth': 5, 'n_estimators': 10}
Mean score: 0.9255249343832022, parameters: {'bootstrap': True, 'max_depth': 5, 'n_estimators': 50}
Mean score: 0.9261811023622047, parameters: {'bootstrap': True, 'max_depth': 5, 'n_estimators': 100}
Mean score: 0.9255249343832022, parameters: {'bootstrap': True, 'max_depth': 5, 'n_estimators': 200}
Mean score: 0.9173228346456693, parameters: {'bootstrap': True, 'max_depth': 10, 'n_estimators': 10}
Mean score: 0.9251968503937008, parameters: {'bootstrap': True, 'max_depth': 10, 'n_estimators': 50}
Mean score: 0.9248687664041996, parameters: {'bootstrap': True, 'max_depth': 10, 'n_estimators': 100}
Mean score: 0.9255249343832022, parameters: {'bootstrap': True, 'max_depth': 10, 'n_estimators': 200}
Mean score: 0.9173228346456693, parameters: {'bootstrap': True, 'max_depth': 15, 'n_estimators': 10}
Mean score: 0.9238845144356955, parameters: {'bootstrap': True, 'max_depth': 15, 'n_estimat

In [51]:
#best param
grid_search.best_params_

{'bootstrap': False, 'max_depth': 5, 'n_estimators': 100}

In [52]:
#using the best parameter model and make predictions
model = grid_search.best_estimator_

predictions = model.predict(X_test_scaled)



In [54]:
#get the accuracy_score
from sklearn.metrics import accuracy_score

score = accuracy_score(y_test, predictions)
print(score)


0.926509186351706


In [55]:
from sklearn.metrics import classification_report

print("classification report")
print(classification_report(y_test, predictions))

classification report
              precision    recall  f1-score   support

       False       0.91      0.95      0.93       412
        True       0.94      0.89      0.92       350

    accuracy                           0.93       762
   macro avg       0.93      0.92      0.93       762
weighted avg       0.93      0.93      0.93       762



In [56]:
import joblib

joblib.dump(model, 'random_fores_rice_model.pkl')

['random_fores_rice_model.pkl']